# Assignment 3 Info


## Class Notes
- Use BeautifulSoup (4) for html scraping
  - https://realpython.com/python-web-scraping-practical-introduction/
  - https://realpython.com/beautiful-soup-web-scraper-python/#step-1-inspect-your-data-source
  - `.prettify()` can be useful to look at the source code
- To join absolute and reltative pages: use module `urlparse`
  - `urlparse.urljoin(url1, url2)`
- To actually *get* the page, you should use the module `requests`
  - `page = requests.get(URL)`
  - `soup = BeautifulSoup(page.content , "html.parser"`
- Process:
  - How ma

## Summary
- **Deadline**: 12 April 2022 18:00
-  Questions:
    1. Are there couples among the employees. If so, who? Are they still together?
       1. Look if they go on holidays together
       2. Look if they changed surnames
       3. Look if they had children
    2. Did any of the employees have a child? If so, who?
       1. 9+ month "break"
       2. If the mother leaves for 9+ months, then the father might also take a leave
       3. They might decrease how much they work (frequency of publish)
    3. If you would be looking to work for Tabularazor Inc., how many holidays can you expect to get per year?
       1. Look at all periods that are larger than "working but not publishing", but shorter than "we're having a baby"

## Group 14
- Antonio Sanchez Martin - ''5245834''
- Felix Unger - ''5673631''
- Jeroen van Paassen - ''4720970''
- Yunus Emre Torlak - '' 5597668''

In [4]:
website = 'https://jdestefani.github.io/SEN163A-TabularRazorArchives/'

In [5]:
import requests
import bs4 as bs

page = requests.get(website)
soup = bs.BeautifulSoup(page.content)
print(soup.prettify())

<head>
 <title>
  Articles
 </title>
</head>
<body>
 <h1>
  All articles
 </h1>
 <br/>
 <br/>
 <h2>
  Articles year 2012
 </h2>
 <br/>
 <div class="yearlink">
  <a href="./2012.html">
   Articles in 2012
  </a>
 </div>
 <h2>
  Articles year 2013
 </h2>
 <br/>
 <div class="yearlink">
  <a href="./2013.html">
   Articles in 2013
  </a>
 </div>
 <h2>
  Articles year 2014
 </h2>
 <br/>
 <div class="yearlink">
  <a href="./2014.html">
   Articles in 2014
  </a>
 </div>
 <h2>
  Articles year 2015
 </h2>
 <br/>
 <div class="yearlink">
  <a href="./2015.html">
   Articles in 2015
  </a>
 </div>
 <h2>
  Articles year 2016
 </h2>
 <br/>
 <div class="yearlink">
  <a href="./2016.html">
   Articles in 2016
  </a>
 </div>
 <h2>
  Articles year 2017
 </h2>
 <br/>
 <div class="yearlink">
  <a href="./2017.html">
   Articles in 2017
  </a>
 </div>
 <h2>
  Articles year 2018
 </h2>
 <br/>
 <div class="yearlink">
  <a href="./2018.html">
   Articles in 2018
  </a>
 </div>
 <h2>
  Articles year 2019
 </h